In [1]:
import tensorflow as tf
from tensorflow import keras

In [27]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error)
    return tf.where(is_small_error, squered_loss, linear_loss)

huber_keras = keras.losses.Huber()

def create_huber(thr=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < thr
        squared_loss = tf.square(error) / 2
        linear_loss = thr * tf.abs(error) - thr**2 / 2
        return tf.where(is_small_error, squered_loss, linear_loss)
    return huber_fn

#### Huber 로스 정의 예시
회귀 모델을 학습할 때 학습 데이터셋에 잡읍 데이터가 조금 있다고 가정해보자.  
평균 제곱 오차는 큰 오차에 너무 과한 벌칙을 가하기 때문에 정확한 모델이 만들어지기 어려움.  
평균 절댓값 오차는 이상치에 관대해서 학습이 수렴하기 오래 걸림.  
> 이럴 경우 **후버 손실**을 사용하면 좋음.

위의 방법보다 더 정석적이고 그럴듯한 방법이 아래 있음

---
## 사용자 정의 손실 함수


In [7]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, thr=1.0, **kwargs):
        self.thr = thr
        super().__init__(**kwargs)
        
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.thr
        squared_loss = tf.squre(error) / 2
        linear_loss = tf.abs(error) * self.thr - self.thr**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 
               "thr": self.thr}
    
# model.compile(loss=HuberLoss(thr=2.0), optimizer="nadam")

# model = keras.models.load_model("my_model_with_costom_loss.h5",
#                                   custom_objects={"HuberLoss": HuberLoss})

---
## 사용자 정의 활성 함수, 초기화, 규제, 제한

In [9]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shapr[0] + shapr[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))
    
def my_positive_weights(weights):    # tf.nn.relu(weights)와 반환값이 같음
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [11]:
layer = keras.layers.Dense(30, activation=my_softplus,
                          kernel_initializer=my_glorot_initializer,
                          kernel_regularizer=my_l1_regularizer,
                          kernel_constraint=my_positive_weights)

이렇게 간단하게 함수를 만들어서 사용할 수 있음.  
만약 **저장될 때 모델과 함께 저장해야 할 하이퍼파라미터를 갖고 있다면** 위와같이 클래스를 상속받아 새로운 클래스를 만들어야 함.  
- keras.regularizers.Regularizer
- keras.constraints.Constraint
- keras.initializers.Initializer
- keras.layers.Layer  (활성함수의 경우)

In [13]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    
    def get_config(self):
        return {"factor": self.factor}

- 층, 모델의 경우 : **call()** 메서드 구현
- 규제, 초기회, 제한의 경우 : **__call__()** 메서드 구현

---
## 사용자 정의 지표
- **손실** : 모델을 학습하기 위해 경사 하강법에서 사용됨. 즉 미분 가능해야 하고 그레디언트가 모든 곳에서 0이 아니어야 함. 사람이 쉽게 이해할 수 없어도 괜찮음.  

- **지표** : 모델을 평가할 때 사용됨. 사람이 이해하기 쉬워야 함. 미분이 가능하지 않거나 모든 곳에서 그레디언트가 0이어도 괜찮음  

> 사실 앞서 만든 손실을 지표로 사용해도 전혀 문제 없음.

In [28]:
# model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0), ])

In [18]:
precision = keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [19]:
precision = keras.metrics.Precision()
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [20]:
precision = keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

위에 **Precision** 클래스를 보면 지표를 계산할 때 앞서 계산한 결과를 버리지 않고 저장했다가 전체 정밀도를 계산함.  
> 배치마다 점진적으로 업데이트되기 때문에 이를 **스트리밍 지표** 라고 함.

**result()** 메서드를 호출하여 현재 지푯값을 얻을 수 있음.  
**variables()** 속성을 사용하여 (진짜 양성과 거짓 양성을 기록한) 변수를 확인할 수도 있음.  
**reset_status()** 메서드를 사용해 이 변수를 초기화할 수도 있음.

In [21]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [23]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [26]:
precision.reset_states()
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [29]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, thr=1.0, **kwargs):
        super().__init__(**kwargs)
        self.thr = thr
        self.huber_fn = create_huber(thr)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
        
    def update_state(self, y_true, y_pred, sample_weights=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
        
    def result(self):
        return self.total / self.count
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config,
               "thr": self.thr}

스트리밍 지표를 만들고 싶다면 **keras.metrics.Metric** 클래스를 상속하면 됨.
- 생성자는 **add_weight()** 메서드를 사용해 여러 배치에 걸쳐 지표의 상태를 기록하기 위한 변수를 만듦.(total, count)
- **update_state** 메서드는 이 클래스를 함수처럼 사용할 때 호출됨. 배치의 레이블과 예측을 바탕으로 변수를 업데이트함 (이 예시에서는 없지만 샘플가중치도 고려해야 함)
- **result** 메서드는 최종 결과를 계산하고 반환함. 이 예에서는 모든 샘플에 대한 평균 후버 손실값임. 이 지표를 함수처럼 사용하면 먼저 update_state()가 호출되고 그 다음 result()가 호출되어 출력이 반환됨
- **get_config** 메서드를 구현하여 thr변수를 모델과 함께 저장함.
- **reset_config** 메서드는 기본적으로 모든 변수를 0.0으로 초기화함. (필요하면 이 함수를 재정의할 수 있음)

---
## 사용자 정의 층
동일한 층 블럭이 여러 번 반복되는 네트워크를 만들 경우 각각의 층 블럭을 하나의 층으로 다루는 것이 편리함.  
먼저 keras.layers.Flatten이나 keras.layers.ReLU와 같은 층은 가중치가 없음.  
> 가중치가 필요 없는 사용자 정의 층을 만들기 위해서 가장 간단한 방법은 **파이썬 함수를 만든 후 keras.layers.Lambda 층으로 감싸는 것**

In [30]:
exp_layer = keras.layers.Lambda(lambda x: tf.exp(x))

>이렇게 만든 사용자 정의 층을 시퀀셜 API나 함수형 API, 서브클래싱 API에서 보통의 층과 동일하게 사용할 수 있음.

지수함수는 이따금 회귀모델에서 예측값의 스케일이 매우 다를 때 출력층에 사용됨.  
위와 같은 방식말고 활성 함수로 사용할 수도 있음. (**activation=tf.exp, activation=keras.activations.exponential, activation="exponential")**

### 상태가 있는 층(즉, 가중치를 가진 층)을 만들려면 
>**keras.layers.Layer**를 상속해야 함

In [32]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
    
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(name="kernel",
                                     shape=[batch_input_shape[-1], self.units],
                                     initializer="glorot_normal")
        self.bias = self.add_weight(name="bias",
                                   shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)
        
    def call(self, X):
        return self.activation(X @ self.kernel+self.bias)
    
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[: -1] + [self.units])
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config,
               "units": self.units,
               "activation": keras.activations.serialize(self.activation)}

- **build()** : 가중치마다 add_weight() 메서드를 호출하여 층의 변수를 만듦. 층이 처음 사용될 때 호출됨. 그리고 **반드시 끝에서 부모의 build() 메서드를 호출해야 함.** 이를 통해 층이 만들어졌다는 것을 케라스가 인식하여 self.built=True로 설정함.
- **call()** : 이 층에 필요한 연산을 수행함. 이 경우 입력 X와 층의 커널을 행렬곱하고 편향을 더함.
- **compute_output_shape** : 이 층의 출력 크기를 반환. 이 예에서는 마지막 차원을 제외하고 입력과 크기가 같음. 마지막 차원은 이 층의 뉴런 개수임. tf.keras에서 크기는 tf.TensorShape클래스의 객체임. 이 객체는 as_list()메서드를 통해 파이썬 리스트로 바뀔 수 있음.
- **get_config** : 앞서 봤던 것과 같음. **keras.activations.serialize()** 를 사용해 활성함수의 전체 설정을 저장함.

---

In [33]:
class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        x1, x2 = X
        return [x1+x2, x1*x2, x1/x2]
    
    def compute_output_shape(self, batch_input_shape):
        b1, b2 = batch_input_shape
        return [b1, b1, b1]

### 여러 입력을 받는(concatenate와 같은) 층을 만들려면  

>**call()** 메서드에 모든 입력이 포함된 튜플을 매개변수 값으로 전달해야 함.  
**compute_output_shape()** 에도 각 입력의 배치 크기를 담은 튜플을 전달해야 함.

위에 만든 층은 하나의 입력과 하나의 출력만 가진 층이 아니기 때문에 **시퀀셜 API**에 사용하진 못함. 대신 함수형 API나 서브클래싱 API에 사용가능.

---

In [34]:
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
        
    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X
    
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

### 학습과 테스트에서 다르게 동작하는 층을 만들려면
>**call()** 메서드에 **trainign** 매개변수를 추가하여 학습인지 테스트인지 결정해야 함.